In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser

import requests 
import pymongo
import pandas as pd
import re

C:\Users\dutch\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


# NASA Mars News

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [3]:
url_mars = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url_mars)

In [4]:
html = browser.html
soup = bs(html, 'html.parser')
# print(soup.prettify())

In [5]:
# Iterate through each article found
# Tried this as well: response = requests.get(url_mars)
# But unles JavaScript is turned of for this website, 'press release' articles show up first, not the latest one (discussed with John and Bobby)
articles = soup.find_all('div', class_='list_text')

news_title = soup.find('div', class_='content_title').text
news_body = soup.find('div', class_='article_teaser_body').text

print("news_title: " + news_title)
print("news_p: " + news_body)

news_title: NASA's Mars 2020 Rover Closer to Getting Its Name
news_p: 155 students from across the U.S. have been chosen as semifinalists in NASA's essay contest to name the Mars 2020 rover, and see it launch from Cape Canaveral this July.


In [6]:
# Remove this from script.py, but keep it here for future reference

# get the first 4 results

limit = 3
print('-----THROUGH SPLINTER------')

for index, article in enumerate(articles):
    title = article.find('div', class_='content_title')
    body = article.find('div', class_='article_teaser_body')
    print('-----------')
    print(index)
    print(title.text)
    news_title = title.text
    print(body.text)
    news_p = body.text
    if index == limit:
        break
        
# The code below could also be used, but since the page uses JavaScript to organize it's dropdown list, 
# It won't return the latest news articles (but the ones under 'Press Release') unless JavaScript is turned of for the website.
 
# Retrieve page with the requests module
response = requests.get(url_mars)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')
# print(soup)

results = soup.find_all('div', class_="slide")

print("    ")
print("    ")

# Get the first 4 results
limit = 3
print('-----THROUGH REQUEST.GET------')

for index, r in enumerate(results):
    title = r.find('div', class_="content_title")
    body = r.find('div', class_="rollover_description_inner")
    print("--------------------------------------------")
    print(index)
    print(title.text.strip())
    print(body.text.strip())
    if index == limit:
        break

-----THROUGH SPLINTER------
-----------
0
NASA's Mars 2020 Rover Closer to Getting Its Name
155 students from across the U.S. have been chosen as semifinalists in NASA's essay contest to name the Mars 2020 rover, and see it launch from Cape Canaveral this July.
-----------
1
Media Get a Close-Up of NASA's Mars 2020 Rover
The clean room at NASA's Jet Propulsion Laboratory was open to the media to see NASA's next Mars explorer before it leaves for Florida in preparation for a summertime launch.
-----------
2
Space History Is Made in This NASA Robot Factory
From rockets to rovers, JPL's Spacecraft Assembly Facility has been at the center of robotic spaceflight. Here's a closer look at what makes it so special.
-----------
3
NASA's Mars 2020 Rover Completes Its First Drive
In a 10-plus-hour marathon, the rover steered, turned and drove in 3-foot (1-meter) increments over small ramps.
    
    
-----THROUGH REQUEST.GET------
--------------------------------------------
0
NASA's Mars 2020 Ro

# JPL Mars Space Images - Featured Image

In [7]:
# URL of page to be scraped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')
# print(soup)

In [8]:
# <img src="/spaceimages/images/mediumsize/PIA00063_ip.jpg" class="fancybox-image" style="display: inline;">
space_image = soup.article.a['data-fancybox-href']
print(space_image)

# unless the background image is needed:
space_image_background = soup.article['style']

split_str = str.split(space_image_background, ' url(\'')
split_str = split_str[1]
split_str = str.split(split_str,')\'')
space_image_background = split_str[0]

print(space_image_background)

/spaceimages/images/mediumsize/PIA16715_ip.jpg
/spaceimages/images/wallpaper/PIA16715-1920x1200.jpg');


In [9]:
#This may have changed in the mean time... only a mediumsize jpg on website
featured_image_url = 'https://www.jpl.nasa.gov' + space_image
print("featured_image_url: " + featured_image_url)

featured_image_url: https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16715_ip.jpg


# Mars Weather

In [10]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)# Mars Weather

In [11]:
# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [12]:
mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
print("mars_weather: " + mars_weather)

mars_weather: If Jacobie wins the contest, he and his family will get to go watch the Mars rover launch from Cape Canaveral this summer! 

H/T @rtphokiehttps://10.wsls.com/38iijbf 


# Mars Facts

In [13]:
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
facts = pd.DataFrame(tables[0])
facts.columns = ['Mars Profile:','']
facts = facts.set_index('Mars Profile:')
facts

,
Mars Profile:,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [14]:
facts.to_html("mars_facts.html")

# Mars Hemispheres

In [15]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [16]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [17]:
html = browser.html
soup = bs(html, 'html.parser')
# print(soup.prettify())

In [18]:
# Find all links to clicked to obtain large image
results = soup.find_all('a', class_='itemLink product-item')
collection = []
for r in results:
    if r.h3:
#         print(r['href'])
        url = "https://astrogeology.usgs.gov" + r['href']
        collection.append(url)

# collection

In [19]:
# for each item in collections - find the large image by viewing the href. Then append to image list.

image_list = []
image_dict = {}

for c in collection:
    print("Now processing: " + c)
    browser.visit(c)
    
    html = browser.html
    soup = bs(html, 'html.parser')
    
    image_url = soup.li.a['href']
    title = soup.h2.text
   
    image_dict = {'hem_titel':title , 'hem_image_url':image_url}
    image_list.append(image_dict)
    
    browser.back

Now processing: https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
Now processing: https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
Now processing: https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
Now processing: https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced


In [20]:
# Show the image_list
image_list

[{'hem_titel': 'Cerberus Hemisphere Enhanced',
  'hem_image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'hem_titel': 'Schiaparelli Hemisphere Enhanced',
  'hem_image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'hem_titel': 'Syrtis Major Hemisphere Enhanced',
  'hem_image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'hem_titel': 'Valles Marineris Hemisphere Enhanced',
  'hem_image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [21]:
return_dict = {}

return_dict['news_title'] = news_title
return_dict['news_body'] = news_body
return_dict['featured_image_url'] = featured_image_url
return_dict['mars_weather'] = mars_weather
return_dict['mars_facts_url'] = 'mars_facts.html'
return_dict['image_list'] = image_list
 
return_dict

{'news_title': "NASA's Mars 2020 Rover Completes Its First Drive",
 'news_body': "155 students from across the U.S. have been chosen as semifinalists in NASA's essay contest to name the Mars 2020 rover, and see it launch from Cape Canaveral this July.",
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16715_ip.jpg',
 'mars_weather': 'If Jacobie wins the contest, he and his family will get to go watch the Mars rover launch from Cape Canaveral this summer! \n\nH/T @rtphokiehttps://10.wsls.com/38iijbf\xa0',
 'mars_facts_url': 'mars_facts.html',
 'image_list': [{'hem_titel': 'Cerberus Hemisphere Enhanced',
   'hem_image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
  {'hem_titel': 'Schiaparelli Hemisphere Enhanced',
   'hem_image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
  {'hem_titel': 'Syrtis Major Hemisphere Enhanced',
   'hem_image_url